In [7]:
import os
from datetime import datetime

import pandas as pd
from notion_client import Client

In [9]:
notion = Client(auth="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")

responseDf = pd.DataFrame(
    columns=databaseColumns,
)

startCursor = None

In [ ]:
def notionObject2DataFrame(notionObject):
    data = {}
    for key, value in notionObject["properties"].items():
        if value["type"] == "title":
            data[key] = value["title"][0]["plain_text"]
        elif value["type"] == "select":
            data[key] = value["select"]["name"]
        elif value["type"] == "number":
            data[key] = value["number"]
        elif value["type"] == "date":
            data[key] = value["date"]["start"]
        elif value["type"] == "relation":
            data[key] = notion.pages.retrieve(value["relation"][0]["id"])
            # print data[key]
            data[key] = [relation["id"] for relation in value["relation"]]
        elif value["type"] == "formula":
            if "string" in value["formula"]:
                data[key] = value["formula"]["string"]
            elif "number" in value["formula"]:
                data[key] = value["formula"]["number"]
    return data


databaseColumns = [
    "Name",
    "Map",
    # "Article Code",
    # "Item Description",
    # "UoM",
    # "Dispatched Qty",
    # "Rate",
    # "Total Amount",
    # "Date",
    # "Location",
    # "Invoice No",
    # "Invoice Version",
]

In [ ]:
while True:
    response = notion.databases.query(
        database_id="24945ba7974f8014a238e9f78855c273",
        # database_id="24845ba7974f808a94e9c967f4113269",
        # filter={"property": "Date", "date": {"equals": "2025-05-15"}},
        start_cursor=startCursor,
    )
    for result in response["results"]:
        responseDf = pd.concat(
            [responseDf, pd.DataFrame([notionObject2DataFrame(result)])],
            ignore_index=True,
        )
    if response.get("has_more"):
        startCursor = response["next_cursor"]
    else:
        break

In [24]:
# response["results"]
responseDf

,Name,Map
0,Major Lazer,"{'object': 'page', 'id': '24945ba7-974f-80e3-9..."
1,Lean On,"{'object': 'page', 'id': '24945ba7-974f-8002-a..."
2,Major Lazer,[24945ba7-974f-80e3-95ae-dc9848bb90ef]
3,Lean On,[24945ba7-974f-8002-ad47-da5fca0cb4af]


In [ ]:
pageId = "24945ba7974f8043b422dac622593f52"
result = notionObject2DataFrame(
    notion.pages.retrieve(
        page_id=pageId,
        # properties={
        #     "Name": {"title": [{"text": {"content": "Updated Name"}}]},
        #     "Map": {"relation": [{"text": {"content": "Updated Map"}}]},
        # },
    )
)

In [18]:
result

,Code,Store Id,Retailer,Location
0,AN,1403419,Rajidi,Ayodhya Nagar


In [16]:
df1 = pd.DataFrame(
    {
        "Article Code": ["A001", "A002", "A003", "A005"],
        "Item Description": ["Item 1", "Item 2", "Item 3", "Item 5"],
        "UoM": ["kg", "kg", "kg", "kg"],
    }
)
df2 = pd.DataFrame(
    {
        "Article Code": ["A001", "A002", "A003", "A004"],
        "Item Description": ["Item 1", "Item 2", "Item 3", "Item 4"],
        "UoM": ["kg", "kg", "kg", "kg"],
    }
)

In [21]:
mrg = df1.merge(df2, how="right", indicator=True)
mrg[mrg["_merge"] == "right_only"].drop(columns=["_merge"])

,Article Code,Item Description,UoM
3,A004,Item 4,kg
